### Introduction: Context & Quick Overview
The American Sign Language letter database of hand gestures represent a multi-class problem with 24 classes of letters (excluding J and Z which require motion).

In [2]:
import os
from pathlib import Path

import pandas as pd
from PIL import Image
from fastai.vision.all import *

In [3]:
def get_images(df):
    IMG_WIDTH =28
    IMG_HEIGHT=28
    
    df['img'] = df[df.columns[df.columns.str.startswith('pixel')]].apply(
        lambda x: PILImage(Image.fromarray(np.array(x.values).reshape((IMG_WIDTH,IMG_HEIGHT)).astype(np.uint8))),
        axis=1
    )
    
    return df[df.columns[[not x for x in df.columns.str.startswith('pixel')]]]

In [4]:
path = Path("../data/raw")

In [5]:
train_df = pd.read_csv(path/"sign_mnist_train.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/sign_mnist_train.csv'

In [10]:
train_df = get_images(train_df)

In [11]:
train_df.head()

,label,img
0,3,PILImage mode=L size=28x28
1,6,PILImage mode=L size=28x28
2,2,PILImage mode=L size=28x28
3,2,PILImage mode=L size=28x28
4,13,PILImage mode=L size=28x28


In [12]:
def get_x(r): return r['img']
def get_y(r): return r['label']
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock), get_x = get_x, get_y=get_y)
dls = dblock.dataloaders(train_df)

In [14]:
learn = vision_learner(dls, resnet34, metrics=error_rate).to_fp16()
learn.fine_tune(20)

In [1]:
test_df = pd.read_csv("../data/raw/sign_mnist_test.csv")
test_df['ImageId'] = test_df.index + 1
test_df = get_images(test_df)
test_df.head()

NameError: name 'pd' is not defined

In [16]:
dl = learn.dls.test_dl(list(test_df['img']))
inp, preds, _, dec_preds = learn.get_preds(dl=dl, with_input=True,with_decoded=True)
test_df['Label'] = dec_preds

In [17]:
test_df.head()

,label,ImageId,img,Label
0,6,1,PILImage mode=L size=28x28,6
1,5,2,PILImage mode=L size=28x28,5
2,10,3,PILImage mode=L size=28x28,9
3,0,4,PILImage mode=L size=28x28,0
4,3,5,PILImage mode=L size=28x28,3


In [18]:
unique_labels = test_df['Label'].unique()
print(unique_labels)

[ 6  5  9  0  3 20 13 19  7  8 11  4 21  2 14 12 18 17 22 16 15 10  1 23]
